In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
rf_model = pd.read_csv("data/tx_user.csv")
rf_model.head()

,Unnamed: 0,userID,itemID,date,Recency,Frequency,MinPurchaseDate,MaxPurchaseDate,NextPurchaseDay,RecencyCluster,FrequencyCluster,OverallScore,Segment,NextPurchaseDayRange
0,0,0,1505,2020-09-01,90,1,2020-09-01,2020-09-01,0,1,0,1,Low-Value,0
1,1,0,12468,2020-08-03,119,1,2020-08-03,2020-08-03,0,1,0,1,Low-Value,0
2,2,0,12505,2020-08-18,104,1,2020-08-18,2020-08-18,0,1,0,1,Low-Value,0
3,3,0,15083,2020-08-03,119,1,2020-08-03,2020-08-03,0,1,0,1,Low-Value,0
4,4,1,4770,2020-09-01,90,1,2020-09-01,2020-09-01,0,1,0,1,Low-Value,0


In [3]:
# drop column 'Unnamed: 0'
rf_model = rf_model.drop(['Unnamed: 0'], axis=1)

In [4]:
#understand the timeline of the data set 
print(min(rf_model['date']))
print(max(rf_model['date']))
# => can be considered the train set to learn the different models

2020-06-01
2020-11-30


In [5]:
x_train = rf_model.drop(['NextPurchaseDayRange'], axis=1)
y_train = rf_model['NextPurchaseDayRange']

In [6]:
#what is our tests set from the task? Predict Dec and Jan 
sub_dec = pd.read_csv("data/submission_dec.csv", delimiter="|")
sub_dec = sub_dec.drop('prediction', axis=1)
sub_jan = pd.read_csv("data/submission_jan.csv", delimiter="|")
sub_jan = sub_jan.drop('prediction', axis=1)

In [28]:
test_jan = pd.read_csv("data/orders.csv", delimiter ="|")

print(min(test_jan['date']))
print(max(test_jan['date']))


2020-06-01
2021-01-31


In [ ]:
"""
DISCLAIMER: I did 2 runs for creating the test sets for dec and jan. The latest was for jan. So please do not wonder
why for variable test_dec the timespan is january. 
"""

In [29]:
#ValueError: X has 2 features, but RandomForestClassifier is expecting 14 features as input
# => generate own test set

test_dec = pd.read_csv("data/orders.csv", delimiter ="|")

print(min(test_dec['date']))
print(max(test_dec['date']))

test_dec= test_dec[(test_dec['date']<'2021-01-29') & (test_dec['date']>='2021-01-01')]

test_dec.set_index(['userID', 'itemID']).index.is_unique
test_dec= test_dec[['userID', 'itemID', 'date']]
test_dec['data'] = pd.to_datetime(test_dec['date'], format='%Y-%m-%d')

#create a dataframe with customer id and first purchase date in test_dec
tx_min_purchase = test_dec.groupby(['userID', 'itemID']).date.min().reset_index()
tx_min_purchase.columns = ['userID', 'itemID','MinPurchaseDate']
tx_min_purchase['MinPurchaseDate'] = pd.to_datetime(tx_min_purchase['MinPurchaseDate'], format='%Y-%m-%d')

#create a dataframe with customer id and last purchase date in test_dec
tx_max_purchase = test_dec.groupby(['userID', 'itemID']).date.max().reset_index()
tx_max_purchase.columns = ['userID', 'itemID','MaxPurchaseDate']
tx_max_purchase['MaxPurchaseDate'] = pd.to_datetime(tx_max_purchase['MaxPurchaseDate'], format='%Y-%m-%d')

#merge two dataframes min and max purchase date
tx_purchase_dates = pd.merge(tx_min_purchase, tx_max_purchase, on=['userID', 'itemID'], how='left')

#calculate the time difference in days:
tx_purchase_dates['NextPurchaseDay'] = (tx_purchase_dates['MaxPurchaseDate'] - tx_purchase_dates['MinPurchaseDate']).dt.days
#calculate recency
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days

## For Frequency
tx_frequency = test_dec.groupby(['userID', 'itemID']).date.count().reset_index()
tx_frequency.columns = ['userID','itemID', 'Frequency'];



#merge all into test_dec
test_dec = pd.merge(test_dec, tx_max_purchase, on=['userID', 'itemID'])
test_dec = pd.merge(test_dec, tx_frequency, on=['userID', 'itemID'])
test_dec = pd.merge(test_dec, tx_purchase_dates, on=['userID', 'itemID'])



2020-06-01
2021-01-31


In [30]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [31]:
#Clustering Recency, Frequency, and Monetary Value:
from sklearn.cluster import KMeans
#clustering for Recency
kmeans = KMeans(n_clusters=4)
kmeans.fit(test_dec[['Recency']])
test_dec['RecencyCluster'] = kmeans.predict(test_dec[['Recency']])

test_dec = order_cluster('RecencyCluster', 'Recency',test_dec,False)

In [32]:
#clustering for Frequency
kmeans = KMeans(n_clusters=4)
kmeans.fit(test_dec[['Frequency']])
test_dec['FrequencyCluster'] = kmeans.predict(test_dec[['Frequency']])

test_dec = order_cluster('FrequencyCluster', 'Frequency',test_dec,True)

print("Clustering of Frequency and Recency") 

display(test_dec)

#We will include new columns “OverallScore” and “Segment

#building overall segmentation
test_dec['OverallScore'] = test_dec['RecencyCluster'] + test_dec['FrequencyCluster']

Clustering of Frequency and Recency


,userID,itemID,date,data,MaxPurchaseDate_x,Recency,Frequency,MinPurchaseDate,MaxPurchaseDate_y,NextPurchaseDay,RecencyCluster,FrequencyCluster
0,39342,25263,2021-01-01,2021-01-01,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0
1,1453,16059,2021-01-01,2021-01-01,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0
2,7153,13675,2021-01-01,2021-01-01,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0
3,7153,18796,2021-01-01,2021-01-01,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0
4,7153,13791,2021-01-01,2021-01-01,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
145674,389,31354,2021-01-25,2021-01-25,2021-01-25,3,4,2021-01-13,2021-01-25,12,3,3
145675,2551,13523,2021-01-10,2021-01-10,2021-01-15,13,4,2021-01-10,2021-01-15,5,2,3
145676,2551,13523,2021-01-10,2021-01-10,2021-01-15,13,4,2021-01-10,2021-01-15,5,2,3
145677,2551,13523,2021-01-10,2021-01-10,2021-01-15,13,4,2021-01-10,2021-01-15,5,2,3


In [33]:
test_dec['Segment'] = 'Low-Value'
test_dec.loc[test_dec['OverallScore']>2,'Segment'] = 'Mid-Value' 
test_dec.loc[test_dec['OverallScore']>4,'Segment'] = 'High-Value'

In [34]:
#Enter the week in which a user will re-order
test_dec['NextPurchaseDayRange'] = 0  ## 0
test_dec.loc[test_dec.NextPurchaseDay>1,'NextPurchaseDayRange'] = 1 ## 2 weeks
test_dec.loc[test_dec.NextPurchaseDay>7,'NextPurchaseDayRange'] = 2 ## 3 weeks
test_dec.loc[test_dec.NextPurchaseDay>14,'NextPurchaseDayRange'] = 3 # 4 weekds
test_dec.loc[test_dec.NextPurchaseDay>21,'NextPurchaseDayRange'] = 4 # 4 weekds

In [35]:
#drop dublicates and rename them
test_dec = test_dec.drop(columns=['MaxPurchaseDate_x', 'data'])
test_dec = test_dec.rename(columns={"MaxPurchaseDate_y": "MaxPurchaseDate"})
test_dec.head()


,userID,itemID,date,Recency,Frequency,MinPurchaseDate,MaxPurchaseDate,NextPurchaseDay,RecencyCluster,FrequencyCluster,OverallScore,Segment,NextPurchaseDayRange
0,39342,25263,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0,0,Low-Value,0
1,1453,16059,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0,0,Low-Value,0
2,7153,13675,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0,0,Low-Value,0
3,7153,18796,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0,0,Low-Value,0
4,7153,13791,2021-01-01,27,1,2021-01-01,2021-01-01,0,0,0,0,Low-Value,0


In [36]:
test_dec.to_csv("data/testset_jan.csv")

In [37]:
#One hot encode test_dec segment
one_hot = pd.get_dummies(test_dec['Segment'])
# Drop column B as it is now encoded
test_dec = test_dec.drop('Segment',axis = 1)
# Join the encoded df
test_dec = test_dec.join(one_hot)


In [38]:
test_dec.to_csv("data/testset_jan_one_hot.csv")

In [19]:
#Invalid columns error for xgb boost:date, MinPurchaseDate, MaxPurchaseDate, Segment
#=> have to preprocess them into int, float or cat value

#transform dates into number of days since 2020-06-01

x_train_1 = x_train

#transform dates into actual dates
x_train_1['date'] = pd.to_datetime(x_train_1['date'])
x_train_1['MinPurchaseDate'] = pd.to_datetime(x_train_1['MinPurchaseDate'])
x_train_1['MaxPurchaseDate']= pd.to_datetime(x_train_1['MaxPurchaseDate'])




In [20]:
x_train_1.to_csv("data/training_set.csv")

In [23]:
x_train_1.dtypes

userID                       int64
itemID                       int64
date                datetime64[ns]
Recency                      int64
Frequency                    int64
MinPurchaseDate     datetime64[ns]
MaxPurchaseDate     datetime64[ns]
NextPurchaseDay              int64
RecencyCluster               int64
FrequencyCluster             int64
OverallScore                 int64
Segment                     object
dtype: object

In [25]:
#no of days passed since 2020-06-01 to get int values for dates
fromDate = min(x_train['date'])
x_train_1['date'] = (x_train_1['date'] - fromDate).dt.days.astype(int)
x_train_1['MinPurchaseDate'] = (x_train_1['MinPurchaseDate'] - fromDate).dt.days.astype(int)
x_train_1['MaxPurchaseDate']= (x_train_1['MaxPurchaseDate'] - fromDate).dt.days.astype(int)

#one hot encoder for segment
one_hot = pd.get_dummies(x_train_1['Segment'])
x_train_1 = x_train_1.drop('Segment',axis = 1)
x_train_1 = x_train_1.join(one_hot)

In [26]:
x_train_1.to_csv("data/training_set_encoded.csv")